# Capture trends 

In [148]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_all_elements_located
from time import sleep
import os
from tqdm import tqdm

In [138]:
URL = 'https://archive.twitter-trending.com/turkey/'
FIRST_DATE = datetime(2022, 1, 1)
LAST_DATE = datetime(2023, 1, 29)

In [138]:
def format_date(date):
    day = str(date.day).zfill(2)
    month = str(date.month).zfill(2)
    year = str(date.year)
    return f'{day}-{month}-{year}'

In [139]:
curr_date = FIRST_DATE


while curr_date <= LAST_DATE:

    curr_date_str = format_date(curr_date)
    print(curr_date_str)

    driver = webdriver.Chrome()
    driver.get(f'{URL}{curr_date_str}')

    sleep(7)
    content = driver.page_source

    driver.quit()

    soup = BeautifulSoup(content, "html.parser")
    tables = soup.find_all("div", class_="tek_tablo")

    df_trend = pd.DataFrame()

    for i in tables:
        col_time = i.find('div', class_='trend_baslik611').text

        trends = []
        for j in i.find_all('tr', class_='tr_table'):
            j = j.text.split(')')[1]

            trends.append(j)

        time_trends = pd.DataFrame(trends, columns=[col_time])
        df_trend = pd.concat([df_trend, time_trends], axis=1)

    df_trend.to_csv(f'trends/{curr_date_str}.csv', index=False)
    
    curr_date += timedelta(days=1)

01-01-2022
02-01-2022
03-01-2022
04-01-2022
05-01-2022
06-01-2022
07-01-2022
08-01-2022
09-01-2022
10-01-2022
11-01-2022
12-01-2022
13-01-2022
14-01-2022
15-01-2022
16-01-2022
17-01-2022
18-01-2022
19-01-2022
20-01-2022
21-01-2022
22-01-2022
23-01-2022
24-01-2022
25-01-2022
26-01-2022
27-01-2022
28-01-2022
29-01-2022
30-01-2022
31-01-2022
01-02-2022
02-02-2022
03-02-2022
04-02-2022
05-02-2022
06-02-2022
07-02-2022
08-02-2022
09-02-2022
10-02-2022
11-02-2022
12-02-2022
13-02-2022
14-02-2022
15-02-2022
16-02-2022
17-02-2022
18-02-2022
19-02-2022
20-02-2022
21-02-2022
22-02-2022
23-02-2022
24-02-2022
25-02-2022
26-02-2022
27-02-2022
28-02-2022
01-03-2022
02-03-2022
03-03-2022
04-03-2022
05-03-2022
06-03-2022
07-03-2022
08-03-2022
09-03-2022
10-03-2022
11-03-2022
12-03-2022
13-03-2022
14-03-2022
15-03-2022
16-03-2022
17-03-2022
18-03-2022
19-03-2022
20-03-2022
21-03-2022
22-03-2022
23-03-2022
24-03-2022
25-03-2022
26-03-2022
27-03-2022
28-03-2022
29-03-2022
30-03-2022
31-03-2022
01-04-2022

In [141]:
all_trends = os.listdir('trends/')

In [172]:
df_all_trends = pd.DataFrame()

for i in tqdm(all_trends):
    
    trend_day = pd.read_csv(f'trends/{i}')
    trend_day = trend_day.melt(value_vars=trend_day.columns, value_name='trend_topic', var_name='time')
    trend_day['date'] = i.replace('.csv', '')
    
    df_all_trends = pd.concat([df_all_trends, trend_day])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 394/394 [00:09<00:00, 40.45it/s]


In [173]:
df_all_trends.shape

(188970, 3)

In [184]:
df_all_trends.head()

,time,trend_topic,date
0,00:10,#Zamlar,01-01-2022
1,00:10,#2022NewYear,01-01-2022
2,00:10,#elektrik,01-01-2022
3,00:10,#BurasıislamDevleti,01-01-2022
4,00:10,#millipiyango,01-01-2022


In [185]:
df_all_trends.to_csv('trends/df_all_trends.csv', index=False)